In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
path = "data/all_M5_series.parquet"

df = pd.read_parquet(path)
df.head(66)

,unique_id,ds,y
0,RTM,2011-01,88163
1,RTM,2011-02,726375
2,RTM,2011-03,763567
3,RTM,2011-04,737713
4,RTM,2011-05,719562
5,RTM,2011-06,753380
6,RTM,2011-07,819820
7,RTM,2011-08,825694
8,RTM,2011-09,813683
9,RTM,2011-10,898243


In [4]:
ids = df['unique_id'].unique().tolist()
print(ids)

['RTM', 'CA', 'TX', 'WI', 'CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3', 'FOODS', 'HOBBIES', 'HOUSEHOLD', 'FOODS_1_001_CA_1_evaluation', 'FOODS_1_001_CA_2_evaluation', 'FOODS_1_001_CA_3_evaluation', 'FOODS_1_001_CA_4_evaluation', 'FOODS_1_001_TX_1_evaluation', 'FOODS_1_001_TX_2_evaluation', 'FOODS_1_001_TX_3_evaluation', 'FOODS_1_001_WI_1_evaluation', 'FOODS_1_001_WI_2_evaluation', 'FOODS_1_001_WI_3_evaluation', 'FOODS_1_002_CA_1_evaluation', 'FOODS_1_002_CA_2_evaluation', 'FOODS_1_002_CA_3_evaluation', 'FOODS_1_002_CA_4_evaluation', 'FOODS_1_002_TX_1_evaluation', 'FOODS_1_002_TX_2_evaluation', 'FOODS_1_002_TX_3_evaluation', 'FOODS_1_002_WI_1_evaluation', 'FOODS_1_002_WI_2_evaluation', 'FOODS_1_002_WI_3_evaluation', 'FOODS_1_003_CA_1_evaluation', 'FOODS_1_003_CA_2_evaluation', 'FOODS_1_003_CA_3_evaluation', 'FOODS_1_003_CA_4_evaluation', 'FOODS_1_003_TX_1_evaluation', 'FOODS_1_003_TX_2_evaluation', 'FOODS_1_003_TX_3_evaluation', 'FOODS_1_003_WI_1_evalua

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982955 entries, 0 to 1982954
Data columns (total 3 columns):
 #   Column     Dtype    
---  ------     -----    
 0   unique_id  object   
 1   ds         period[M]
 2   y          int64    
dtypes: int64(1), object(1), period[M](1)
memory usage: 45.4+ MB


In [6]:
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive
from hierarchicalforecast.utils import aggregate
import dask.dataframe as dd
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut

d:\ferramentas\Pandas\Lib\site-packages\statsforecast\core.py:27: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
from mlforecast import MLForecast
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from mlforecast.target_transforms import Differences
from statsforecast.models import AutoARIMA, ETS, Theta
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

In [8]:
df['ds'] = df['ds'].dt.to_timestamp()

In [9]:
top_10_itens = ['FOODS_3_090_CA_3_evaluation', 'FOODS_3_586_TX_2_evaluation', 'FOODS_3_586_TX_3_evaluation', 'FOODS_3_586_CA_3_evaluation',
                'FOODS_3_090_CA_1_evaluation', 'FOODS_3_090_WI_3_evaluation', 'FOODS_3_090_TX_2_evaluation', 'FOODS_3_090_TX_3_evaluation',
                'FOODS_3_252_TX_2_evaluation', 'FOODS_3_586_TX_1_evaluation', 'HOBBIES_1_234_CA_3_evaluation', 'HOBBIES_1_348_CA_1_evaluation',
                'HOBBIES_1_371_CA_1_evaluation', 'HOBBIES_1_254_CA_3_evaluation', 'HOBBIES_1_371_CA_3_evaluation', 'HOBBIES_1_348_CA_3_evaluation',
                'HOBBIES_1_268_CA_1_evaluation', 'HOBBIES_1_341_CA_3_evaluation', 'HOBBIES_1_015_CA_3_evaluation', 'HOBBIES_1_268_CA_3_evaluation',
                'HOUSEHOLD_1_118_CA_3_evaluation', 'HOUSEHOLD_1_459_CA_3_evaluation', 'HOUSEHOLD_1_334_CA_3_evaluation', 'HOUSEHOLD_1_303_CA_3_evaluation',
                'HOUSEHOLD_1_521_CA_3_evaluation', 'HOUSEHOLD_1_459_WI_2_evaluation', 'HOUSEHOLD_1_465_CA_3_evaluation', 'HOUSEHOLD_1_351_CA_3_evaluation',
                'HOUSEHOLD_1_019_CA_3_evaluation', 'HOUSEHOLD_1_110_WI_2_evaluation']

bottom_10_itens = ['FOODS_3_778_CA_2_evaluation', 'FOODS_2_057_WI_2_evaluation', 'FOODS_2_071_TX_3_evaluation', 'FOODS_2_209_TX_2_evaluation',
                'FOODS_1_079_CA_2_evaluation', 'FOODS_2_337_CA_2_evaluation', 'FOODS_2_117_CA_4_evaluation', 'FOODS_2_209_CA_4_evaluation',
                'FOODS_3_171_TX_3_evaluation', 'FOODS_2_158_CA_2_evaluation', 'HOBBIES_1_170_WI_3_evaluation', 'HOBBIES_1_026_CA_1_evaluation',
                'HOBBIES_1_212_WI_2_evaluation', 'HOBBIES_1_063_CA_4_evaluation', 'HOBBIES_2_025_TX_1_evaluation', 'HOBBIES_2_025_CA_4_evaluation',
                'HOBBIES_1_402_WI_1_evaluation', 'HOBBIES_1_230_CA_4_evaluation', 'HOBBIES_1_217_TX_1_evaluation', 'HOBBIES_1_186_WI_3_evaluation',
                'HOUSEHOLD_2_216_CA_4_evaluation', 'HOUSEHOLD_1_378_CA_1_evaluation', 'HOUSEHOLD_1_020_CA_3_evaluation', 'HOUSEHOLD_2_307_CA_4_evaluation',
                'HOUSEHOLD_1_336_CA_3_evaluation', 'HOUSEHOLD_2_130_WI_2_evaluation', 'HOUSEHOLD_1_512_CA_4_evaluation', 'HOUSEHOLD_1_073_CA_3_evaluation',
                'HOUSEHOLD_2_307_WI_2_evaluation', 'HOUSEHOLD_2_101_WI_2_evaluation']

In [10]:
ids = df['unique_id'].unique().tolist()[:17]
ids.extend(top_10_itens)
ids.extend(bottom_10_itens)

In [11]:
def forecasting_with_statsforecast(data, unique_id):
    subset = data[data['unique_id'] == unique_id]
    test_size = (int)(subset.shape[0]*0.2)
    
    Y_test_df = subset.tail(test_size)
    Y_train_df = subset.drop(Y_test_df.index)

    models = [AutoARIMA(season_length=12), ETS(season_length=12), Theta(season_length=12)]

    sf = StatsForecast(df=Y_train_df, models=models, freq='MS', n_jobs=-1)
    sf.fit()

    test_df = sf.forecast(h=len(Y_test_df))
    
    return test_df

In [12]:
tests_dfs = []

for id in df['unique_id'].unique().tolist()[:17]:
    test = forecasting_with_statsforecast(df, id)
    test['unique_id'] = id
    tests_dfs.append(test)

combined_test_df = pd.concat(tests_dfs, ignore_index=True)

d:\ferramentas\Pandas\Lib\site-packages\statsforecast\models.py:880: FutureWarning: `ETS` will be deprecated in future versions of `StatsForecast`. Please use `AutoETS` instead.
  ETS._warn()
d:\ferramentas\Pandas\Lib\site-packages\statsforecast\core.py:467: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
d:\ferramentas\Pandas\Lib\site-packages\statsforecast\core.py:467: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
d:\ferramentas\Pandas\Lib\site-packages\statsforecast\core.py:467: FutureWarning: The `df` argument of the StatsFo

In [13]:
combined_test_df

,ds,AutoARIMA,ETS,Theta,unique_id
0,2015-05-01,1.089440e+06,1.092574e+06,1.100728e+06,RTM
1,2015-06-01,1.095476e+06,1.092672e+06,1.105017e+06,RTM
2,2015-07-01,1.110348e+06,1.092757e+06,1.109306e+06,RTM
3,2015-08-01,1.121872e+06,1.092831e+06,1.113596e+06,RTM
4,2015-09-01,1.087772e+06,1.092897e+06,1.117886e+06,RTM
5,2015-10-01,1.092998e+06,1.092954e+06,1.122175e+06,RTM
6,2015-11-01,1.074467e+06,1.093004e+06,1.126464e+06,RTM
7,2015-12-01,1.061058e+06,1.093048e+06,1.130754e+06,RTM
8,2016-01-01,1.085884e+06,1.093087e+06,1.135044e+06,RTM
9,2016-02-01,1.067624e+06,1.093121e+06,1.139333e+06,RTM


In [14]:
def forecasting_with_mlforecast(data, unique_id):
    subset = data[data['unique_id'] == unique_id]

    test_size = (int)(subset.shape[0]*0.2)
    
    Y_test_df = subset.tail(test_size)
    Y_train_df = subset.drop(Y_test_df.index)

    models = [LGBMRegressor(), XGBRegressor(), LinearRegression()]
    mlf = MLForecast(
        models=models,
        lags=[1, 2, 3, 4, 5, 6],
        lag_transforms={
            1: [ExpandingMean()],
            6: [RollingMean(window_size=6)]
        },
        date_features=['month', 'year'],
        target_transforms=[Differences([1])],
        freq='M'
    )

    mlf.fit(Y_train_df[['unique_id', 'ds', 'y']])
    test_df = mlf.predict(h=len(Y_test_df))
    test_df['ds'] = pd.date_range(start=Y_test_df['ds'].min(), periods=len(test_df), freq='M')
    test_df['unique_id'] = unique_id

    return test_df

In [15]:
tests_dfs = []

for id in df['unique_id'].unique().tolist()[:17]:
    test = forecasting_with_mlforecast(df, id)
    test['unique_id'] = id
    tests_dfs.append(test)

combined_test_df_ml = pd.concat(tests_dfs, ignore_index=True)

d:\ferramentas\Pandas\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] O sistema não pode encontrar o arquivo especificado
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "d:\ferramentas\Pandas\Lib\site-packages\joblib\externals\loky\backend\context.py", line 199, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "d:\ferramentas\Pandas\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\ferramentas\Pandas\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "d:\ferramentas\Pandas\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000085 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 45
[LightGBM] [Info] Number of data points in the train set: 40, number of used features: 3
[LightGBM] [Info] Start training from score 5153.275000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [16]:
combined_test_df_ml

,unique_id,ds,LGBMRegressor,XGBRegressor,LinearRegression
0,RTM,2015-05-31,1.106569e+06,1.082644e+06,1057877.00
1,RTM,2015-06-30,1.103736e+06,1.155445e+06,1104831.00
2,RTM,2015-07-31,1.116875e+06,1.171345e+06,1052655.00
3,RTM,2015-08-31,1.114042e+06,1.208477e+06,1077961.00
4,RTM,2015-09-30,1.127182e+06,1.203941e+06,1062362.00
5,RTM,2015-10-31,1.124348e+06,1.151112e+06,1034649.00
6,RTM,2015-11-30,1.137488e+06,1.147457e+06,1046253.00
7,RTM,2015-12-31,1.134655e+06,1.144285e+06,992539.00
8,RTM,2016-01-31,1.147795e+06,1.101566e+06,1004719.00
9,RTM,2016-02-29,1.144962e+06,1.139015e+06,1011119.00


In [17]:
combined_test_df['LGBMRegressor'] = combined_test_df_ml['LGBMRegressor']
combined_test_df['XGBRegressor'] = combined_test_df_ml['XGBRegressor']
combined_test_df['LinearRegression'] = combined_test_df_ml['LinearRegression']

In [18]:
var = combined_test_df_ml[['LGBMRegressor', 'XGBRegressor', 'LinearRegression']].equals(combined_test_df[['LGBMRegressor', 'XGBRegressor', 'LinearRegression']])
print(var)

True


In [19]:
combined_test_df.to_parquet('tests.parquet')